<a href="https://colab.research.google.com/github/youngjooo/DCC-project/blob/main/face_detection_with_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



#function ClickConnect() {var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); buttons.forEach(function(btn) { btn.click(); }); console.log("1분마다 자동 재연결"); document.querySelector("colab-toolbar-button#connect").click(); } setInterval(ClickConnect,1000*60);
# ==============================================================================

# Object Detection


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/s?q=google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1%20OR%20google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub models</a>
  </td>
</table>

This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Setup


In [ ]:
#@title Imports and function definitions

# For running inference on the TF-Hub module.
import tensorflow as tf
import cv2
from google.colab.patches import cv2_imshow
import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2.5.0
The following GPU devices are available: /device:GPU:0


## Example use

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [ ]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  global tmp_url

  tmp_url = url

  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  
  #이미지 높이 조절하기
 # pil_image = pil_image.resize((int(pil_image.width), int(pil_image.height - 500)))
  #pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  print(pil_image.size)
  width, height = pil_image.size
  pil_image = ImageOps.fit(pil_image, (int(width) , int(height-130)), Image.ANTIALIAS)
  
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

## Apply module

Load a public image from Open Images v4, save locally, and display.

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
#구글 드라이브에 마운트 해주세요 
#마운트 후 드라이브에 저장된 분류.csv파일을 보내주시면 될 것 같습니다. 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
final_url_lists = []

In [ ]:
not_final_url_lists = []

In [ ]:

def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}
  #
  # print("Found %d objects." % len(result["detection_scores"]))
  # print("Inference time: ", end_time-start_time)

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  correct = 'c'
  count = 0
  for i in range(len(result["detection_class_entities"]) - 1):
      # display_image(image_with_boxes)
      # print((result["detection_class_entities"][i]))
      if (str(result["detection_class_entities"][i]) == 'b\'Human face\''):
          # if (str(result["detection_class_entities"][i])=='b\'Human eye\''):
          count = count + 1
          # print(result["detection_class_entities"][i])
          # print(result["detection_scores"][i])
          # print(result["detection_scores"][i])
          if (result["detection_scores"][i] > 0.65) and count >= 1:
              if count>1 and (result["detection_scores"][i] > 0.65):
                  count=count+1
                  continue
              # print(i, "번째")
              final_url_lists.append(tmp_url)
            #  display_image(image_with_boxes)
              correct = 'correct'

  if correct == 'c':
      not_final_url_lists.append(tmp_url)



In [ ]:
import csv

#img 파일 경로 조심해주세요!
#아마 구글 드라이브에 넣고 경로 복사하시면 될 것 같아요 
read_url=open('/content/drive/MyDrive/Colab Notebooks/img_data_url.csv').readlines() # 파일을 한 줄씩 전체를 읽어들여서 리스트로 반환


### More images
Perform inference on some additional images with time tracking.


In [ ]:
from PIL import ImageFile 
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def detect_img(image_url):
  # start_time = time.time()
  image_path = download_and_resize_image(image_url, 640, 480)
  run_detector(detector, image_path)
  # end_time = time.time()
  # print("Inference time:",end_time-start_time) 

In [ ]:
# detect_img(image_urls[0])
# print(read_url[1])
#만장씩 해서 돌려주시면 감사하겠습니다. 

# 109999, 120000 (115999, 117999)
for i in range(117999, 200000):
  detect_img(read_url[i])

(1960, 4032)
Image downloaded to /tmp/tmpxhio99b0.jpg.
(840, 1260)
Image downloaded to /tmp/tmp25dv_evq.jpg.
(720, 960)
Image downloaded to /tmp/tmpvfql6auu.jpg.
(790, 840)
Image downloaded to /tmp/tmpp13qczt2.jpg.
(2992, 2992)
Image downloaded to /tmp/tmp4okysuou.jpg.
(640, 640)
Image downloaded to /tmp/tmp54ncwqpv.jpg.
(2056, 3088)
Image downloaded to /tmp/tmpqyf87grg.jpg.
(800, 847)
Image downloaded to /tmp/tmpde9ch3cd.jpg.
(720, 960)
Image downloaded to /tmp/tmpga5k7o16.jpg.
(720, 960)
Image downloaded to /tmp/tmpn2wn0t0e.jpg.
(1000, 1000)
Image downloaded to /tmp/tmpxoapz5h7.jpg.
(860, 860)
Image downloaded to /tmp/tmpgesoynzf.jpg.
(1000, 1000)
Image downloaded to /tmp/tmpizxs3w1j.jpg.
(1000, 1000)
Image downloaded to /tmp/tmpedc1khcg.jpg.
(720, 960)
Image downloaded to /tmp/tmpl7tu_sm4.jpg.
(1440, 1440)
Image downloaded to /tmp/tmpwe80khyf.jpg.
(910, 910)
Image downloaded to /tmp/tmp8ffhwdw5.jpg.
(2160, 2880)
Image downloaded to /tmp/tmp6rxgkllo.jpg.
(2056, 3088)
Image downloaded

In [ ]:
#face detection 된 url을 csv로 변환

#경로랑 파일명 조심 
import pandas as pd

# 저장할 경로 입니다. 
final_url=pd.DataFrame(final_url_lists)
final_url.to_csv('/content/drive/MyDrive/Colab Notebooks/face_urln.csv',header=False,index=False)

In [ ]:
#face detection 안 된 url을 csv로 변환
#파일명 
not_final_url=pd.DataFrame(not_final_url_lists)
not_final_url.to_csv('/content/drive/MyDrive/Colab Notebooks/not_face_urln.csv',header=False,index=False)